****Caricamento test.jsonl e train.jsonl****

In [20]:
import pandas as pd
import numpy as np
import json
import warnings
from tqdm.auto import tqdm

# --- 1. Import di Scikit-learn ---
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression   # <-- Il tuo modello
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler # <-- StandardScaler è incluso
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.exceptions import ConvergenceWarning

In [21]:
try:
    train_df = pd.read_json('/kaggle/input/fds-pokemon-battles-prediction-2025/train.jsonl', lines=True)
    test_df = pd.read_json('/kaggle/input/fds-pokemon-battles-prediction-2025/test.jsonl', lines=True)
    
    # Convertiamo subito il target in 1 (Vittoria) e 0 (Sconfitta)
    if 'player_won' in train_df.columns:
        train_df['player_won'] = train_df['player_won'].astype(int)
    print("Caricamento riuscito!")
except Exception as e:
    print(f"Errore nel caricamento dati: {e}")

#####
riga_da_rimuovere = 4877

# Usiamo un controllo per sicurezza, nel caso la riga non esista
if riga_da_rimuovere in train_df.index:
    train_df = train_df.drop(riga_da_rimuovere)
    print(f"Riga {riga_da_rimuovere} rimossa con successo.")
else:
    print(f"Riga {riga_da_rimuovere} non trovata (forse già rimossa o non presente).")

filtro_livello_100 = train_df['p1_team_details'].apply(
    lambda team_list: all(pokemon.get('level') == 100 for pokemon in team_list)
)

train_df = train_df[filtro_livello_100]

Caricamento riuscito!
Riga 4877 rimossa con successo.


In [22]:
# Mappa di efficacia dei tipi per la Generazione 1
# Nota: 'Special' in Gen 1 copre sia Atk Sp. che Def Sp.
# Non ci sono tipi Dark, Steel, o Fairy.
TYPE_CHART_GEN1 = {
    'NORMAL': {'ROCK': 0.5, 'GHOST': 0.0},
    'FIRE': {'FIRE': 0.5, 'WATER': 0.5, 'GRASS': 2.0, 'ICE': 2.0, 'BUG': 2.0, 'ROCK': 0.5},
    'WATER': {'FIRE': 2.0, 'WATER': 0.5, 'GRASS': 0.5, 'GROUND': 2.0, 'ROCK': 2.0, 'DRAGON': 0.5},
    'ELECTRIC': {'WATER': 2.0, 'ELECTRIC': 0.5, 'GRASS': 0.5, 'GROUND': 0.0, 'FLYING': 2.0, 'DRAGON': 0.5},
    'GRASS': {'FIRE': 0.5, 'WATER': 2.0, 'ELECTRIC': 1.0, 'GRASS': 0.5, 'POISON': 0.5, 'GROUND': 2.0, 'FLYING': 0.5, 'BUG': 0.5, 'ROCK': 2.0, 'DRAGON': 0.5},
    'ICE': {'WATER': 0.5, 'GRASS': 2.0, 'ICE': 0.5, 'GROUND': 2.0, 'FLYING': 2.0, 'DRAGON': 2.0},
    'FIGHTING': {'NORMAL': 2.0, 'POISON': 0.5, 'FLYING': 0.5, 'PSYCHIC': 0.5, 'BUG': 0.5, 'ROCK': 2.0, 'GHOST': 0.0},
    'POISON': {'GRASS': 2.0, 'POISON': 0.5, 'GROUND': 0.5, 'BUG': 2.0, 'ROCK': 0.5, 'GHOST': 0.5},
    'GROUND': {'FIRE': 2.0, 'ELECTRIC': 2.0, 'GRASS': 0.5, 'POISON': 2.0, 'FLYING': 0.0, 'BUG': 0.5, 'ROCK': 2.0},
    'FLYING': {'ELECTRIC': 0.5, 'GRASS': 2.0, 'FIGHTING': 2.0, 'BUG': 2.0, 'ROCK': 0.5},
    'PSYCHIC': {'FIGHTING': 2.0, 'POISON': 2.0, 'PSYCHIC': 0.5, 'GHOST': 1.0}, # In Gen 1, Psychic era immune a Ghost per un bug, ma i dati Showdown potrebbero averlo corretto. Assumiamo 1.0 per sicurezza, o 0.0 se il bug è emulato. Qui usiamo 1.0.
    'BUG': {'FIRE': 0.5, 'GRASS': 2.0, 'FIGHTING': 0.5, 'POISON': 2.0, 'FLYING': 0.5, 'PSYCHIC': 2.0},
    'ROCK': {'FIRE': 2.0, 'ICE': 2.0, 'FIGHTING': 0.5, 'GROUND': 0.5, 'FLYING': 2.0, 'BUG': 2.0},
    'GHOST': {'NORMAL': 0.0, 'PSYCHIC': 0.0, 'GHOST': 2.0}, # Famoso bug: Lick (Ghost) non colpisce Psychic.
    'DRAGON': {'DRAGON': 2.0},
}

# Funzione helper per calcolare l'efficacia
def get_type_effectiveness(move_type, target_types):
    if move_type not in TYPE_CHART_GEN1:
        return 1.0
    
    multiplier = 1.0
    chart_for_move = TYPE_CHART_GEN1[move_type]
    
    for target_type in target_types:
        if target_type in chart_for_move:
            multiplier *= chart_for_move[target_type]
            
    return multiplier

# Pokémon dominanti nel metagame Gen 1 OU (S-Tier e A-Tier)
# La loro presenza è un segnale fortissimo.
META_THREATS_GEN1 = {
    'Snorlax', 'Tauros', 'Chansey', 'Alakazam', 'Starmie', 'Exeggutor', 
    'Zapdos', 'Jolteon', 'Rhydon', 'Golem', 'Lapras'
}

# Mosse di setup o status chiave
STATUS_MOVES = {'Thunder Wave', 'Sleep Powder', 'Sing', 'Toxic', 'Lovely Kiss', 'Spore', 'Stun Spore', 'Glare'}
SETUP_MOVES = {'Amnesia', 'Swords Dance', 'Agility', 'Growth'}

In [23]:
def create_advanced_features(df):
    processed_data = []
    for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="Creazione features"):
        p1_team = row['p1_team_details']
        p2_lead = row['p2_lead_details']
        timeline = row['battle_timeline']
        p1_lead = p1_team[0]
        
        feat_lead_speed_diff = p1_lead['base_spe'] - p2_lead['base_spe']
        
        p1_seen_status = {p['name']: {'hp_pct': 100, 'status': None} for p in p1_team}
        p2_seen_status = {p2_lead['name']: {'hp_pct': 100, 'status': None}}
        
        feat_end_boost_diff = 0
        feat_num_turns = 0
        
        if timeline:
            feat_num_turns = timeline[-1].get('turn', 0)
            for turn in timeline:
                p1_state = turn.get('p1_pokemon_state')
                if p1_state and p1_state.get('name'):
                    p1_name = p1_state['name']
                    p1_seen_status.setdefault(p1_name, {'hp_pct': 100, 'status': None})
                    p1_seen_status[p1_name]['hp_pct'] = p1_state.get('hp_pct', p1_seen_status[p1_name]['hp_pct'])
                    p1_seen_status[p1_name]['status'] = p1_state.get('status', p1_seen_status[p1_name]['status'])
                    
                p2_state = turn.get('p2_pokemon_state')
                if p2_state and p2_state.get('name'):
                    p2_name = p2_state['name']
                    p2_seen_status.setdefault(p2_name, {'hp_pct': 100, 'status': None})
                    p2_seen_status[p2_name]['hp_pct'] = p2_state.get('hp_pct', p2_seen_status[p2_name]['hp_pct'])
                    p2_seen_status[p2_name]['status'] = p2_state.get('status', p2_seen_status[p2_name]['status'])

                if turn.get('turn') == feat_num_turns:
                    p1_boosts = sum(p1_state.get('boosts', {}).values()) if p1_state else 0
                    p2_boosts = sum(p2_state.get('boosts', {}).values()) if p2_state else 0
                    feat_end_boost_diff = p1_boosts - p2_boosts

        p1_total_hp_seen = sum(p['hp_pct'] for p in p1_seen_status.values())
        p2_total_hp_seen = sum(p['hp_pct'] for p in p2_seen_status.values())
        feat_hp_advantage_seen = p1_total_hp_seen - p2_total_hp_seen
        
        feat_mons_revealed_diff = len(p2_seen_status) - len(p1_seen_status)
        
        p1_team_status_count = sum(1 for p in p1_seen_status.values() if p['status'] is not None)
        p2_team_status_count = sum(1 for p in p2_seen_status.values() if p['status'] is not None)
        feat_team_status_diff = p1_team_status_count - p2_team_status_count # (P1 status) - (P2 status)

        processed_data.append({
            'battle_id': row['battle_id'],
            'p1_lead_name': p1_lead['name'], 'p2_lead_name': p2_lead['name'],
            'lead_speed_diff': feat_lead_speed_diff,
            'hp_advantage_seen': feat_hp_advantage_seen,
            'mons_revealed_diff': feat_mons_revealed_diff,
            'team_status_diff': feat_team_status_diff,
            'end_boost_diff': feat_end_boost_diff,
            'num_turns': feat_num_turns
        })
    return pd.DataFrame(processed_data).set_index('battle_id')

In [24]:
print("Inizio feature engineering avanzata sul set di training...")
X_train_features = create_advanced_features(train_df)

print("\nInizio feature engineering avanzata sul set di test...")
X_test_features = create_advanced_features(test_df)

# Definiamo la nostra variabile target 'y'
y_train = train_df.set_index('battle_id')['player_won']

# Allineiamo X e y
y_train = y_train.loc[X_train_features.index]

print("\nFeature engineering completato. Esempio di dati trasformati:")
print(X_train_features.head())

Inizio feature engineering avanzata sul set di training...


Creazione features:   0%|          | 0/9996 [00:00<?, ?it/s]


Inizio feature engineering avanzata sul set di test...


Creazione features:   0%|          | 0/5000 [00:00<?, ?it/s]


Feature engineering completato. Esempio di dati trasformati:
          p1_lead_name p2_lead_name  lead_speed_diff  hp_advantage_seen  \
battle_id                                                                 
0              starmie      starmie                0         201.225312   
1                 jynx     alakazam              -25          -0.990000   
2            exeggutor      chansey                5         299.020000   
3               gengar       tauros                0         100.180000   
4             alakazam      starmie                5         100.610000   

           mons_revealed_diff  team_status_diff  end_boost_diff  num_turns  
battle_id                                                                   
0                          -2                 0               0         30  
1                           0                 0               2         30  
2                          -2                -1               0         30  
3                          

In [29]:
numeric_features = [
    'lead_speed_diff',
    'hp_advantage_seen',
    'mons_revealed_diff',
    'team_status_diff',
    'end_boost_diff',
    'num_turns'
]
categorical_features = ['p1_lead_name', 'p2_lead_name']

# Creiamo i trasformatori (StandardScaler e OneHotEncoder)
numeric_transformer = Pipeline(steps=[('scaler', RobustScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

In [32]:

# Dividiamo i dati di training per una validazione locale
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train_features, 
    y_train, 
    test_size=0.25, # 20% per la validazione
    random_state=42,
    stratify=y_train # Mantiene l'equilibrio delle classi
)

print(f"Dimensione Training Split: {X_train_split.shape}")
print(f"Dimensione Validation Split: {X_val_split.shape}")

# 1. Creiamo la pipeline con un modello "di default"
# Usiamo C=1.0 come valore predefinito
baseline_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42, max_iter=1000, solver='liblinear', C=1.0))
])

# 2. Alleniamo il modello base SUL SOLO SET DI TRAINING SPLIT
print("\nAllenamento del modello baseline...")
baseline_pipeline.fit(X_train_split, y_train_split)

# 3. Valutiamo il modello base SUL SET DI VALIDAZIONE
y_val_pred = baseline_pipeline.predict(X_val_split)
val_accuracy = accuracy_score(y_val_split, y_val_pred)

print(f"\n--- Risultati Modello Baseline ---")
print(f"Accuracy sul Validation Set: {val_accuracy:.4f}")
print("---------------------------------")

Dimensione Training Split: (7497, 8)
Dimensione Validation Split: (2499, 8)

Allenamento del modello baseline...

--- Risultati Modello Baseline ---
Accuracy sul Validation Set: 0.7407
---------------------------------


In [37]:
from sklearn.model_selection import GridSearchCV
print("\nAvvio di GridSearchCV per l'ottimizzazione degli iperparametri...")
# 1. Creiamo la pipeline (la stessa di prima, ma senza 'C' definito)
# La pipeline che verrà testata da GridSearchCV
tuning_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42, max_iter=1000, solver='liblinear'))
])

# 2. Definiamo la griglia dei parametri
# Vogliamo testare diversi valori per 'classifier__C'
param_grid = {
    'classifier__penalty': ['l1','l2'], 
    'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'classifier__solver': ['liblinear'] 
}

# 3. Impostiamo GridSearchCV
# cv=5 significa 5-fold cross-validation
# scoring='accuracy' è la nostra metrica
# n_jobs=-1 usa tutti i processori
grid_search = GridSearchCV(
    tuning_pipeline, 
    param_grid, 
    cv=10, 
    scoring='accuracy', 
    n_jobs=-1,
    verbose=1 # Mostra i log
)

# 4. Avviamo la ricerca sull'INTERO set di training
# (GridSearchCV gestirà internamente le divisioni di cross-validation)
grid_search.fit(X_train_features, y_train)

# 5. Analizziamo i risultati
print("\n--- Risultati GridSearchCV ---")
print(f"Migliori parametri trovati: {grid_search.best_params_}")
print(f"Migliore Accuracy (media CV): {grid_search.best_score_:.4f}")
print("------------------------------")


Avvio di GridSearchCV per l'ottimizzazione degli iperparametri...
Fitting 10 folds for each of 14 candidates, totalling 140 fits

--- Risultati GridSearchCV ---
Migliori parametri trovati: {'classifier__C': 1000, 'classifier__penalty': 'l1', 'classifier__solver': 'liblinear'}
Migliore Accuracy (media CV): 0.8277
------------------------------


In [ ]:
import pandas as pd

# 1. Estrai il modello migliore (la pipeline completa) da GridSearchCV
final_model = grid_search.best_estimator_

# 2. Estrai i nomi delle feature numeriche (li abbiamo già)
# Assicurati che 'numeric_features' sia la lista aggiornata che hai usato
# numeric_features = ['lead_speed_diff', 'p1_lead_type_adv', ...]

# 3. Estrai i nomi delle feature categoriche create dal OneHotEncoder
# Questo è il passaggio chiave
try:
    categorical_names = final_model.named_steps['preprocessor'] \
                                     .named_transformers_['cat'] \
                                     .named_steps['onehot'] \
                                     .get_feature_names_out(categorical_features)
except AttributeError:
    # Fallback per versioni più vecchie di scikit-learn
    categorical_names = final_model.named_steps['preprocessor'] \
                                     .named_transformers_['cat'] \
                                     .named_steps['onehot'] \
                                     .get_feature_names_out()

# 4. Combina tutti i nomi delle feature nell'ordine corretto
all_feature_names = numeric_features + list(categorical_names)

# 5. Estrai i coefficienti (l'importanza) dal modello di regressione logistica
coefficients = final_model.named_steps['classifier'].coef_[0]

# 6. Crea un DataFrame per visualizzarli in modo chiaro
importance_df = pd.DataFrame({
    'Feature': all_feature_names,
    'Coefficient': coefficients
})

# 7. Aggiungi il 'Coefficiente Assoluto' per ordinare per impatto (sia positivo che negativo)
importance_df['Impact'] = importance_df['Coefficient'].abs()
importance_df = importance_df.sort_values(by='Impact', ascending=False)

# 8. Stampa i risultati
print("--- Importanza delle Feature (Coefficienti del Modello) ---")
print(importance_df.to_string()) # .to_string() stampa tutto il DataFrame senza troncamenti

In [40]:
# 1. Estrai il modello finale
final_model = grid_search.best_estimator_

# 2. Genera le predizioni (saranno True/False)
test_predictions_bool = final_model.predict(X_test_features)

# 3. --- CORREZIONE 1: Converti True/False in 1/0 ---
# .astype(int) converte True -> 1 e False -> 0
test_predictions_int = test_predictions_bool.astype(int)

# 4. Prendi i battle_id dall'indice
test_battle_ids = X_test_features.index

# 5. Crea il DataFrame con le due colonne CORRETTE
submission_df = pd.DataFrame({
    'battle_id': test_battle_ids,
    'player_won': test_predictions_int  # Usa la versione 1/0
})

# 6. --- CORREZIONE 2: Salva SENZA l'indice di pandas ---
# Aggiungendo 'index=False' si risolve il problema della "stessa colonna".
submission_df.to_csv('submission_predictions.csv', index=False)

print("\n-------------------------------------------------")
print("File 'submission_predictions.csv' creato con successo!")
print("Ora conterrà 1 e 0, e colonne separate.")
print("-------------------------------------------------")

# Stampa un'anteprima
print(submission_df.head())


-------------------------------------------------
File 'submission_predictions.csv' creato con successo!
Ora conterrà 1 e 0, e colonne separate.
-------------------------------------------------
   battle_id  player_won
0          0           0
1          1           1
2          2           1
3          3           1
4          4           1
